In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Импорт модулей

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
import tensorflow.keras.optimizers as O
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

Загрузка данных

In [ ]:
images = np.load('/kaggle/input/sf-captcha-recognition/data/images.npy')
labels = np.load('/kaggle/input/sf-captcha-recognition/data/labels.npy')

print("Train dataset:", images.shape, labels.shape, "Labels are:", set(labels))
print("Picture shape:", images[0].shape)

In [ ]:
submission = pd.read_csv('../input/sf-captcha-recognition/sample_submission.csv')

In [ ]:
cols = 8
rows = 2
fig = plt.figure(figsize=(2 * cols - 1, 2.5 * rows - 1))
for i in range(cols):
    for j in range(rows):
        random_index = np.random.randint(0, len(labels))
        ax = fig.add_subplot(rows, cols, i * rows + j + 1)
        ax.grid('off')
        ax.axis('off')
        ax.imshow(images[random_index])
        ax.set_title(labels[random_index])
plt.show()

In [ ]:
num_classes = 26 #количество классов
img_shape = (48, 48, 3) #размер изображений
TEST_SIZE = 0.25 #размер тестовой выборки
RANDOM_SEED = 42

Нормализуем изображения

In [ ]:
def normalize_pictures(pics):
    return pics.astype('float32')/255 - 0.5

X = normalize_pictures(images)
Y = labels

In [ ]:
print("Shape of X:",X.shape)
print("Shape of Y:",Y.shape)

In [ ]:
images_sub = np.load('../input/sf-captcha-recognition/data/images_sub.npy')
images_sub = normalize_pictures(images_sub)

Задаем архитектуру сети

In [ ]:
def make_model():
    model = Sequential()
    model.add(L.Conv2D(16, kernel_size=3, strides=1, padding='same', activation = 'relu', input_shape=img_shape))
    model.add(BatchNormalization())
    model.add(L.Conv2D(32, kernel_size=3, strides=1, padding='same', activation = 'relu',))
    model.add(BatchNormalization())
    model.add(L.MaxPool2D())
    model.add(L.Dropout(0.25))
    model.add(L.Conv2D(32, kernel_size=3, strides=1, padding='same', activation = 'relu',))
    model.add(BatchNormalization())
    model.add(L.Conv2D(64, kernel_size=3, strides=1, padding='same', activation = 'relu',))
    model.add(BatchNormalization())
    model.add(L.MaxPool2D())
    model.add(L.Dropout(0.25))
    model.add(L.Flatten())
    model.add(L.Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(L.Dropout(0.5))
    model.add(L.Dense(26, activation='softmax'))
    return model

Тренируем модель

In [ ]:
opt = O.Adam(learning_rate=0.005)

def train_model(make_model_func=make_model, optimizer=opt):
  BATCH_SIZE = 48
  EPOCHS = 45

  K.clear_session()
  model = make_model_func()

  model.compile(
      loss='sparse_categorical_crossentropy',
      optimizer=optimizer,
      metrics=['accuracy']
  )

  model.fit(
      X, Y,  # нормализованные данные
      batch_size=BATCH_SIZE,
      validation_split = 0.15,
      epochs=EPOCHS,
      shuffle=False
  )
  
  return model

In [ ]:
labels_sub = train_model().predict(images_sub)

Submission

In [ ]:
submission['Category'] = labels_sub.argmax(axis = 1)

In [ ]:
submission.set_index('Id').to_csv('submission.csv')